In [1]:
import os
import datetime
import pandas as pd
import math
import zipfile
from tqdm import tqdm
from matplotlib import pyplot as plt
from dateutil.relativedelta import relativedelta

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

# Defiine functions

In [3]:
def get_regional_duid_mapping(file_path,regionid):
    # Read the duid mapping table
    # It is obtained from Brian (from AER database)
    # Only SA1 DUIDs, covers both GENERATOR and LOAD
    duid_mapping = pd.read_csv(f'{file_path}Reference_data/duidinfo.csv',
                               usecols=['DUID','REGIONID','STATIONNAME','PARTICIPANTNAME',
                                        'DISPATCHTYPE','SCHEDULE_TYPE','FUELTYPE'])
    duid_mapping = duid_mapping[(duid_mapping['REGIONID']==regionid)]
    return duid_mapping

In [4]:
# # read duid mapping
# def read_regional_standing_data(standing_data_path,regionid):
#     duid_mapping = pd.read_excel(standing_data_path,
#                                  sheet_name='{} Standing Data'.format(regionid[0:-1]),
#                                  usecols=['{} DUID'.format(regionid),
#                                           '{} FUELTYPE'.format(regionid),
#                                           '{} DISPATCHTYPE'.format(regionid),
#                                           '{} SCHEDULE_TYPE'.format(regionid)])\
#                      .rename(columns={'{} DUID'.format(regionid):'DUID',
#                                       '{} FUELTYPE'.format(regionid):'FUELTYPE',
#                                       '{} DISPATCHTYPE'.format(regionid):'DISPATCHTYPE',
#                                       '{} SCHEDULE_TYPE'.format(regionid):'SCHEDULE_TYPE'})
#     duid_mapping['Intermit'] = duid_mapping['FUELTYPE'].apply(lambda x: 'Y' if x in ['Wind','Solar'] else 'N')
    
# #     duid_mapping = duid_mapping[(duid_mapping['DISPATCHTYPE']=='GENERATOR')&
# #                                 (duid_mapping['SCHEDULE_TYPE'].isin(['SCHEDULED','SEMI-SCHEDULED']))]
#     return duid_mapping

In [4]:
def read_dispatchload(file_path,year,month):
    temp_dispatchload = pd.read_csv(f'{file_path}/Raw_data_zipped/DISPATCHLOAD/PUBLIC_DVD_DISPATCHLOAD_{str(year)}{str(month).zfill(2)}010000.zip',
                                    skiprows=1,
                                    usecols=['DUID','INTERVENTION','INITIALMW','SETTLEMENTDATE','TOTALCLEARED'],
                                    parse_dates=['SETTLEMENTDATE']).dropna(subset=['DUID'])
    temp_dispatchload = temp_dispatchload.sort_values(by=['SETTLEMENTDATE','DUID','INTERVENTION'])\
                                         .drop_duplicates(subset=['SETTLEMENTDATE','DUID'],keep='last')
    return temp_dispatchload

In [5]:
def read_genunits(file_path,year,month):
    temp_genunits = pd.read_csv(f'{file_path}/Raw_data_zipped/GENUNITS/PUBLIC_DVD_GENUNITS_{str(year)}{str(month).zfill(2)}010000.zip',
                                skiprows=1,
                                usecols=['GENSETID','GENSETTYPE','CO2E_EMISSIONS_FACTOR','CO2E_ENERGY_SOURCE'])\
                      .dropna(subset=['GENSETID'])
    temp_genunits = temp_genunits[temp_genunits['GENSETTYPE']=='GENERATOR']
    temp_genunits = temp_genunits.rename(columns={'GENSETID':'DUID',
                                                  'CO2E_EMISSIONS_FACTOR':'EMISSIONS_FACTOR',
                                                  'CO2E_ENERGY_SOURCE':'ENERGY_SOURCE'})
    return temp_genunits

In [6]:
def read_and_process_aemo_reg(reg_file_path):
    # Read AEMO Reg doc to fill missing DUIDs
    AEMO_reg = pd.read_excel(reg_file_path,
                             sheet_name='PU and Scheduled Loads',
                             usecols=['DUID','Fuel Source - Descriptor'])\
                 .rename(columns={'Fuel Source - Descriptor':'ENERGY_SOURCE'})\
                 .drop_duplicates(subset=['DUID'])
    return AEMO_reg

In [7]:
def read_rooftop_pv(file_path,year,month):
    temp_rooftop = pd.read_csv(f'{file_path}/Raw_data_zipped/ROOFTOP_PV_ACTUAL/PUBLIC_DVD_ROOFTOP_PV_ACTUAL_{str(year)}{str(month).zfill(2)}010000.zip',
                               skiprows=1,
                               usecols=['INTERVAL_DATETIME','REGIONID','TYPE','POWER'],
                               parse_dates=['INTERVAL_DATETIME'])
    if 'DAILY' in temp_rooftop['TYPE'].unique():
        temp_rooftop = temp_rooftop[(temp_rooftop['REGIONID'].isin(['NSW1','VIC1','QLD1','SA1','TAS1']))&
                                    (temp_rooftop['TYPE']=='DAILY')]
    else:
        temp_rooftop = temp_rooftop[(temp_rooftop['REGIONID'].isin(['NSW1','VIC1','QLD1','SA1','TAS1']))&
                                    (temp_rooftop['TYPE']=='MEASUREMENT')]
    temp_rooftop = temp_rooftop.rename(columns={'INTERVAL_DATETIME':'TI_DATETIME'}).drop(columns=['TYPE'])
    temp_rooftop = temp_rooftop.rename(columns={'POWER':'ROOFTOP_PV'})
    temp_rooftop['ROOFTOP_PV_MWH'] = temp_rooftop['ROOFTOP_PV']/2
    return temp_rooftop

# Sample case

## Data prep

In [8]:
# This is a mapping dict that translates AEMO's reg doc fueltype to energy source in GENUNITS
# It is used to fill in the fueltype (and emission factor) for the missing DUIDs
AEMO_reg_to_genunits_mapping_dict = dict()
AEMO_reg_to_genunits_mapping_dict['Grid'] = 'Battery Storage'
AEMO_reg_to_genunits_mapping_dict['Water'] = 'Hydro'
AEMO_reg_to_genunits_mapping_dict['Solar'] = 'Solar'
AEMO_reg_to_genunits_mapping_dict['Solar '] = 'Solar'
AEMO_reg_to_genunits_mapping_dict['Landfill Methane / Landfill Gas'] = 'Landfill biogas methane'
AEMO_reg_to_genunits_mapping_dict['Diesel'] = 'Diesel oil'
AEMO_reg_to_genunits_mapping_dict['Waste Coal Mine Gas'] = 'Coal mine waste gas'
AEMO_reg_to_genunits_mapping_dict['Wind'] = 'Wind'
AEMO_reg_to_genunits_mapping_dict['Natural Gas'] = 'Natural Gas (Pipeline)'
AEMO_reg_to_genunits_mapping_dict['Black Coal'] = 'Black coal'
AEMO_reg_to_genunits_mapping_dict['Coal Seam Methane'] = 'Coal seam methane'
AEMO_reg_to_genunits_mapping_dict['Bagasse'] = 'Bagasse'
AEMO_reg_to_genunits_mapping_dict['Biogas - sludge'] = 'Other Biofuels'
AEMO_reg_to_genunits_mapping_dict['Solar'] = 'Solar'
AEMO_reg_to_genunits_mapping_dict['Natural Gas / Diesel'] = 'Natural Gas (Pipeline)'
AEMO_reg_to_genunits_mapping_dict['Brown Coal'] = 'Brown coal'
AEMO_reg_to_genunits_mapping_dict['solar'] = 'Solar'
AEMO_reg_to_genunits_mapping_dict['Kerosene'] = 'Kerosene - non aviation'
AEMO_reg_to_genunits_mapping_dict['Sewerage / Waste Water'] = 'Other Biofuels'
AEMO_reg_to_genunits_mapping_dict['Natural Gas / Fuel Oil'] = 'Natural Gas (Pipeline)'
AEMO_reg_to_genunits_mapping_dict['Gas'] = 'Natural Gas (Pipeline)'
AEMO_reg_to_genunits_mapping_dict['Ethane'] = 'Other'

In [9]:
# Put detaied energy source into a broader category
# Allows: Black coal, Brown coal, Gas, Hydro, Renewable, Other
energy_source_broad_mapping_dict = dict()
energy_source_broad_mapping_dict['Black coal'] = 'Black coal'
energy_source_broad_mapping_dict['Brown coal'] = 'Brown coal'
energy_source_broad_mapping_dict['Hydro'] = 'Hydro'
energy_source_broad_mapping_dict['Natural Gas (Pipeline)'] = 'Gas'
energy_source_broad_mapping_dict['Solar'] = 'Renewable'
energy_source_broad_mapping_dict['Wind'] = 'Renewable'
energy_source_broad_mapping_dict['Battery Storage'] = 'Other'
energy_source_broad_mapping_dict['Landfill biogas methane'] = 'Other'
energy_source_broad_mapping_dict['Diesel oil'] = 'Other'
energy_source_broad_mapping_dict['Coal mine waste gas'] = 'Other'
energy_source_broad_mapping_dict['Coal seam methane'] = 'Other'
energy_source_broad_mapping_dict['Bagasse'] = 'Other'
energy_source_broad_mapping_dict['Other Biofuels'] = 'Other'
energy_source_broad_mapping_dict['Kerosene - non aviation'] = 'Other'
energy_source_broad_mapping_dict['Other'] = 'Other'
energy_source_broad_mapping_dict['PV'] = 'Rooftop PV'

In [10]:
file_path = '/Volumes/EnergyData/AER/'
standing_data_path = f'{file_path}Economic_withholding/Reference_data/duidinfo.csv'
reg_file_path = f'{file_path}Reference_data/NEM Registration and Exemption List.xlsx'

In [11]:
# Get YearMonth to FY mapping dict
YearMonthFY = pd.read_excel(f'{file_path}Reference_data/duidinfo_SRMC.xlsx',
                         sheet_name='YearMonthFY').set_index(['Year','Month'])['FY'].to_dict()

In [12]:
# Read and process AEMO reg list
aemo_reg = read_and_process_aemo_reg(reg_file_path)
aemo_reg = aemo_reg[~aemo_reg['ENERGY_SOURCE'].isin([' ','-'])&
                    ~aemo_reg['ENERGY_SOURCE'].isna()]
aemo_reg['ENERGY_SOURCE'] = aemo_reg['ENERGY_SOURCE'].apply(lambda x:AEMO_reg_to_genunits_mapping_dict[x])

In [13]:
# Read duid mapping
duid_mapping = pd.DataFrame()
for regionid in ['NSW1','QLD1','VIC1','SA1','TAS1']:
    duid_mapping = pd.concat([duid_mapping,get_regional_duid_mapping(file_path,regionid)])

In [27]:
def monthly_generation_and_emission(file_path,duid_mapping,aemo_reg,energy_source_broad_mapping_dict,
                                    YearMonthFY,year,month):
    
    def match_energy_source(source):
        if source == 'Gas':
            source_use = 'Natural Gas (Pipeline)'
        elif source == 'Coal-Black':
            source_use = 'Black coal'
        elif source == 'Diesel':
            source_use = 'Diesel oil'
        elif source == 'Liquid':
            source_use = 'Diesel oil'
        else:
            source_use = source
        return source_use
    
    temp_dispatchload = read_dispatchload(file_path,year,month)
    temp_genunits = read_genunits(file_path,year,month)
    temp = temp_dispatchload.merge(right=temp_genunits.drop(columns=['ENERGY_SOURCE']),
                                   on=['DUID'],
                                   how='left')\
                            .merge(right=duid_mapping,
                                   on=['DUID'],
                                   how='left')\
                            .merge(right=aemo_reg,
                                   on=['DUID'],
                                   how='left')
    temp = temp[temp['DISPATCHTYPE']!='LOAD']
    temp = temp[~temp['FUELTYPE'].isna()]
    # Fix duids with no energy source
    duids_to_fix = temp[temp['ENERGY_SOURCE'].isna()]['DUID'].unique()
    temp['ENERGY_SOURCE'] = temp.parallel_apply(lambda row: row['FUELTYPE'] if row['DUID'] in duids_to_fix else row['ENERGY_SOURCE'],axis=1)
    
    # Match the energy sources in GENUNITS
    temp['ENERGY_SOURCE'] = temp['ENERGY_SOURCE'].parallel_apply(lambda x: match_energy_source(x))
    
    # Fill duids with mission emission factor
    duids_to_fill = temp[temp['EMISSIONS_FACTOR'].isna()]['DUID'].unique()
    ave_emission = temp.groupby(by=['ENERGY_SOURCE','DUID'],as_index=False)[['EMISSIONS_FACTOR']]\
                        .mean()\
                        .groupby(by=['ENERGY_SOURCE'])['EMISSIONS_FACTOR']\
                        .mean().to_dict()
    temp['EMISSIONS_FACTOR'] = temp.parallel_apply(lambda row: ave_emission[row['ENERGY_SOURCE']] if row['DUID'] in duids_to_fill else row['EMISSIONS_FACTOR'], axis=1)
    # Calculate generation and emission
    temp['ACTUAL_GEN'] = temp['INITIALMW']/12
    temp['EMISSION'] = temp['EMISSIONS_FACTOR']*temp['ACTUAL_GEN']
    
#     print(temp[temp['DUID'].isin(duids_to_fix)]['ACTUAL_GEN'].sum())
    
    # Shift the timestamp (this way is easier for processing data on a monthly basis)
    temp['SETTLEMENTDATE'] = temp['SETTLEMENTDATE'].parallel_apply(lambda x:x-datetime.timedelta(minutes=5))
    # Clean the formatted data
    temp = temp[(temp['SETTLEMENTDATE']>=datetime.datetime(year,month,1,0,0,0))&
                (temp['SETTLEMENTDATE']<datetime.datetime(year,month,1,0,0,0)+relativedelta(months=1))]\
                .drop_duplicates()
    
    # Return aggregated results
    temp_result = temp.groupby(by=['REGIONID','ENERGY_SOURCE'],as_index=False)[['EMISSION','ACTUAL_GEN']].sum()
    
    # Read and process PV generation
    temp_pv = read_rooftop_pv(file_path,year,month).drop(columns=['ROOFTOP_PV'])\
                                                .rename(columns={'ROOFTOP_PV_MWH':'ACTUAL_GEN'})\
                                                .drop_duplicates()
    temp_pv = temp_pv[(temp_pv['TI_DATETIME']>=datetime.datetime(year,month,1,0,30,0))&
                      (temp_pv['TI_DATETIME']<=datetime.datetime(year,month,1,0,0,0)+relativedelta(months=1))]
    temp_pv = temp_pv.groupby(by=['REGIONID'],as_index=False)[['ACTUAL_GEN']].sum()
    temp_pv['ENERGY_SOURCE'] = 'PV'
    temp_pv['EMISSION'] = 0
    temp_pv = temp_pv[['REGIONID','ENERGY_SOURCE','EMISSION','ACTUAL_GEN']]
    
    # Combine data
    temp_result = pd.concat([temp_result,temp_pv])
    
    # Tag with broader energy source type
    temp_result['FUEL_CAT'] = temp_result['ENERGY_SOURCE'].parallel_apply(lambda x: energy_source_broad_mapping_dict[x])
    
    # Tag year and month
    temp_result['Year'] = year
    temp_result['Month'] = month
    temp_result['FY'] = YearMonthFY[(year,month)]    
    
    return temp_result

In [28]:
# test_2023 = pd.DataFrame()
# for month in range(1,13):
#     temp_result = monthly_generation_and_emission(file_path,duid_mapping,aemo_reg,energy_source_broad_mapping_dict,
#                                                   YearMonthFY,2023,month)
#     test_2023 = pd.concat([test_2023,temp_result])
#     print(month,'Done')

1 Done
2 Done
3 Done
4 Done
5 Done
6 Done
7 Done
8 Done
9 Done
10 Done
11 Done
12 Done


In [29]:
# test_2023[test_2023['ENERGY_SOURCE']!='PV'][['ACTUAL_GEN','EMISSION']].sum()

ACTUAL_GEN    1.827764e+08
EMISSION      1.196763e+08
dtype: float64

In [ ]:
# temp_result

## Monthly data workflow

In [17]:
result_df = pd.DataFrame()
for year in range(2017,2025):
    if year == 2017:
        for month in range(7,13):
            try:
                temp_result = monthly_generation_and_emission(file_path,duid_mapping,aemo_reg,energy_source_broad_mapping_dict,
                                                              YearMonthFY,year,month)
                result_df = pd.concat([result_df,temp_result])
                print(year,month,'Done.')
            except:
                print(year,month,'No data.')
    elif year in range(2018,2025):
        for month in range(1,13):
            try:
                temp_result = monthly_generation_and_emission(file_path,duid_mapping,aemo_reg,energy_source_broad_mapping_dict,
                                                              YearMonthFY,year,month)
                result_df = pd.concat([result_df,temp_result])
                print(year,month,'Done.')
            except:
                print(year,month,'No data.')            

2017 7 Done.
2017 8 Done.
2017 9 Done.
2017 10 Done.
2017 11 Done.
2017 12 Done.
2018 1 Done.
2018 2 Done.
2018 3 Done.
2018 4 Done.
2018 5 Done.
2018 6 Done.
2018 7 Done.
2018 8 Done.
2018 9 Done.
2018 10 Done.
2018 11 Done.
2018 12 Done.
2019 1 Done.
2019 2 Done.
2019 3 Done.
2019 4 Done.
2019 5 Done.
2019 6 Done.
2019 7 Done.
2019 8 Done.
2019 9 Done.
2019 10 Done.
2019 11 Done.
2019 12 Done.
2020 1 Done.
2020 2 Done.
2020 3 Done.
2020 4 Done.
2020 5 Done.
2020 6 Done.
2020 7 Done.
2020 8 Done.
2020 9 Done.
2020 10 Done.
2020 11 Done.
2020 12 Done.
2021 1 Done.
2021 2 Done.
2021 3 Done.
2021 4 Done.
2021 5 Done.
2021 6 Done.
2021 7 Done.
2021 8 Done.
2021 9 Done.
2021 10 Done.
2021 11 Done.
2021 12 Done.
2022 1 Done.
2022 2 Done.
2022 3 Done.
2022 4 Done.
2022 5 Done.
2022 6 Done.
2022 7 Done.
2022 8 Done.
2022 9 Done.
2022 10 Done.
2022 11 Done.
2022 12 Done.
2023 1 Done.
2023 2 Done.
2023 3 Done.
2023 4 Done.
2023 5 Done.
2023 6 Done.
2023 7 Done.
2023 8 Done.
2023 9 Done.
2023 10

In [18]:
result_df.to_csv('/Volumes/System/Users/weisun/OneDrive/AER work/Generation_emission_plotting.csv',index=False)